In [1]:
%run ../Regression_Analysis/SLP_REGRESSION.ipynb

Epoch 1 : loss = 35.182, accuracy = 0.547/0.803
Epoch 2 : loss = 8.874, accuracy = 0.814/0.809
Epoch 3 : loss = 8.134, accuracy = 0.805/0.803
Epoch 4 : loss = 8.017, accuracy = 0.802/0.804
Epoch 5 : loss = 7.924, accuracy = 0.803/0.802
Epoch 6 : loss = 7.843, accuracy = 0.802/0.803
Epoch 7 : loss = 7.767, accuracy = 0.802/0.804
Epoch 8 : loss = 7.697, accuracy = 0.802/0.806
Epoch 9 : loss = 7.634, accuracy = 0.805/0.804
Epoch 10 : loss = 7.574, accuracy = 0.804/0.805

 Final Test : final accuracy = 0.805
[[0.55842942]
 [0.76572744]
 [0.00429812]
 [2.24857982]
 [1.78899292]
 [0.66286655]
 [2.723841  ]
 [0.65809305]
 [0.54706306]
 [1.1225188 ]]
[4.52425871]
Epoch 1 : loss = 7.521, accuracy = 0.803/0.757
Epoch 2 : loss = 6.212, accuracy = 0.817/0.803
Epoch 3 : loss = 5.859, accuracy = 0.824/0.835
Epoch 4 : loss = 5.598, accuracy = 0.828/0.838
Epoch 5 : loss = 5.440, accuracy = 0.830/0.838
Epoch 6 : loss = 5.240, accuracy = 0.834/0.796
Epoch 7 : loss = 5.201, accuracy = 0.834/0.850
Epoch 8

In [2]:
### Function Call Relation
"""
Steel_exec -> load_steel_dataset
           -> init_model
           -> train_and_test -> arrange_data
                             -> get_train_data
                             -> get_test_data
                             -> run_train -> forward_nueralnet
                                          -> forward_postproc -> softmax_cross_entropy_with_logits -> softmax
                                          -> eval_accuracy
                                          -> backprop_neuralnet -> softmax_cross_entropy_with_logits_derv -> softmax
                                          -> backprop_postproc
                             -> run_test -> forward_neuralnet
                                         -> forward_postproc
                                         -> eval_accuracy
"""

'\nSteel_exec -> load_steel_dataset\n           -> init_model\n           -> train_and_test -> arrange_data\n                             -> get_train_data\n                             -> get_test_data\n                             -> run_train -> forward_nueralnet\n                                          -> forward_postproc -> softmax_cross_entropy_with_logits -> softmax\n                                          -> eval_accuracy\n                                          -> backprop_neuralnet -> softmax_cross_entropy_with_logits_derv -> softmax\n                                          -> backprop_postproc\n                             -> run_test -> forward_neuralnet\n                                         -> forward_postproc\n                                         -> eval_accuracy\n'

In [3]:
# define main function(steel_exec)

def steel_exec(epoch_count = 10, mb_size = 10, report = 1):
    load_steel_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)
    

In [4]:
def load_steel_dataset():
    with open('./archive/faults.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    
    input_cnt, output_cnt = 27, 7
    data = np.asarray(rows, dtype = 'float32')
    

In [5]:
# redefine forward postproc and backprop postproc functions

def forward_postproc(output, y):
    entropy = softmax_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
       
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
        
    return G_output

In [6]:
# redefine eval_accuracy function

def eval_accuracy(output, y):
    estimate = np.argmax(output, axis = 1)
    answer = np.argmax(y, axis = 1)
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

In [7]:
# define softmax related functions

def softmax(x):
    max_elem = np.max(x, axis = 1)
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis = 1)
    prods = (exp.transpose() / sum_exp).transpose()
    
    return prods

def softmax_derv(x, y):
    mb_size, nom_size = x.shape
    derv = np.ndarray([mb_size, nom_size, nom_size])
    
    for n in range(mb_size):
        for i in range(nom_size):
            for j in range(nom_size):
                derv[n, i, j] = -y[n, i] * y[n, j]
                
            derv[n, i, i] += y[n, i]
    
    return derv

def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs + 1.0e-10), axis = 1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

In [8]:
def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis = 0)
    
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [9]:
steel_exec()

Epoch 1 : loss = 16.059, accuracy = 0.303/0.432
Epoch 2 : loss = 15.910, accuracy = 0.309/0.223
Epoch 3 : loss = 15.152, accuracy = 0.342/0.294
Epoch 4 : loss = 15.301, accuracy = 0.335/0.215
Epoch 5 : loss = 15.078, accuracy = 0.345/0.187
Epoch 6 : loss = 15.182, accuracy = 0.341/0.332
Epoch 7 : loss = 14.915, accuracy = 0.352/0.223
Epoch 8 : loss = 15.450, accuracy = 0.329/0.437
Epoch 9 : loss = 15.464, accuracy = 0.328/0.187
Epoch 10 : loss = 15.093, accuracy = 0.345/0.330

 Final Test : final accuracy = 0.330


In [10]:
a = np.array([[1, 2, 3],
            [3, 2, 1]])

In [11]:
a.shape

(2, 3)